In [1]:
import pickle
import os

from re import subn
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

from analyzer import data_cleaner
from analyzer.data_transformation import TfidfDataTransformer, BagOfWordsTransformer, DataTransformer

from imblearn.under_sampling import TomekLinks, RandomUnderSampler, CondensedNearestNeighbour,EditedNearestNeighbours
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, ComplementNB
from sklearn.neural_network import MLPClassifier



In [2]:
# Run this cell if you want to ignore warnings
import warnings
from sklearn.exceptions import ConvergenceWarning
from scipy.linalg import LinAlgWarning
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=LinAlgWarning, module='sklearn')
warnings.simplefilter('ignore', category=ConvergenceWarning)


In [3]:
def load_and_clean_data(filepath, rename_dict=None, vectorizer_output='models/vectorizer.sav', **kwargs) -> (pd.DataFrame, DataTransformer):
    # Load data
    df = pd.read_csv(filepath, **kwargs)

    if rename_dict is not None:
        df.rename(columns=rename_dict, inplace=True)
    #display(df.head(5))
    # Clean data
        # Remove @ mentions
    df['clean_text'] = np.vectorize(data_cleaner.clean_mentions)(df['text'])
        # Remove non alfabet chars
    df['clean_text'] = df['clean_text'].str.replace("[^a-zA-Z#]", " ")
        # Remove short words
    df['clean_text'] = df['clean_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
    # Transform
    tr = TfidfDataTransformer()
    # Stemming
    df['clean_text'] = tr.stemming(df['clean_text'])

    tr.vectorizer_fit(df['clean_text'])
    df_tfidf = tr.transform(df['clean_text'])
    pickle.dump(tr.vectorizer, open(vectorizer_output, 'wb'))


    return df, df_tfidf, tr

def balance_data(X, y, balancer = RandomUnderSampler(sampling_strategy='not minority',random_state=1337)) -> pd.DataFrame:
    X_balanced, y_balanced = balancer.fit_resample(X, y)
    return X_balanced, y_balanced



def fit_model(X, y, model_type, params, model_output='../models/model.sav', ):
    model = model_type(**params)
    model.fit(X,y)
    if model_output is not None:
        pickle.dump(model, open(model_output, 'wb'))
    return model

proba_models = set(['LogisticRegression', 'BernoulliNB', 'MLPClassifier', 'ComplementNB'])
def predict(model, data):
    model_name = type(model).__name__
    if model_name in proba_models:
        predictions = model.predict_proba(data)
    else:
        predictions = model.predict(data)
    return predictions

def metrics(predictions, true_values, name_prefix="", plot=True):
    predictions_int =  predictions[:,1]>=0.3
    # Calculate metrics
    metrics = {
        'f1_score': f1_score(true_values, predictions_int),
        'confusion_matrix': confusion_matrix(true_values, predictions_int)
    }
    # Plot?? mby
    if plot:
        ConfusionMatrixDisplay.from_predictions(true_values, predictions_int)
        plt.show()
    # Write to outputfile.
    with open(f'out/results/{name_prefix}_out.out', 'w') as f:
            f.write(str(metrics))
    return metrics

In [4]:
def clean_folders():
    for folder, end in [('models', '.sav'), ('out/results', '.out')]:
        filelist = [ f for f in os.listdir(folder) if f.endswith(end) ]
        for f in filelist:
            os.remove(os.path.join(folder, f))

In [5]:
def __model_tuning__(X_train, y_train, X_valid, y_valid, model_type,params, output_file, data_label="", verbose=False, plot=False):
    output_name = f'{model_type.__name__}-$-{data_label}-{datetime.now().timestamp()}'
    model_output = f'models/{output_name}.sav'
    outputs = []
    if verbose:
        print(f'Starting tuning of {model_type.__name__} with data labeled: {data_label}')
        print('===============')
    parameter_grid = ParameterGrid(params)
    for i, p in enumerate(parameter_grid):
        param_string = subn("[{}',:]","","".join(str(p).split()))[0]
        if verbose:
            print(f'Params {i+1}/{len(parameter_grid)}: {p}')
        model_output = model_output.replace('$',param_string)
        model = fit_model(X_train, y_train, model_type, params=p, model_output=model_output)
        predictions = predict(model, X_valid)
        output = metrics(predictions, y_valid, output_name.replace('$',param_string),plot=plot)
        outputs.append(output)
        if verbose:
            print(f'Output: {output}')
    if verbose:
        print(f'Tuning ended')
        print('===============')
    return outputs


In [17]:
clean_folders()

In [7]:
verbose = True
random_state = 1337

In [8]:
models = [
    (LogisticRegression, {
        'penalty': ['none', 'l2'],
        'class_weight': [None,'balanced'],
        'n_jobs': [-1],
        'solver': ['newton-cholesky']
    }),
    (BernoulliNB, {
        'alpha' : np.arange(0.0, 1.0, 0.2),
        'fit_prior': [True, False],
        'binarize': [None]

    }),
    (MLPClassifier, {
        'hidden_layer_sizes': [(100,), (100, 100), (50, 50, 50), (50,)],
        'activation': ['identity', 'logistic'],
        'alpha': [0.0001, 0.001]
    }),
    (ComplementNB,{
        'alpha' : np.arange(0.0, 1.0, 0.2),
        'fit_prior': [True, False],
        'norm': [True, False],
    })


]


for model, param_grid in models[:2]:
    if verbose:
        print('=====================')
        print(f'=={model.__name__}==')
        print('=====================')

    df, df_tfidf, tr = load_and_clean_data('data/train.csv', rename_dict={'tweet':'text', 'label':'output'})
    X_train, X_valid, y_train, y_valid = train_test_split(df_tfidf, df['output'],test_size=0.3,random_state=random_state)
    X_train_balanced, y_train_balanced = balance_data(X_train, y_train)

    output = __model_tuning__(X_train_balanced, y_train_balanced, X_valid, y_valid, model, param_grid, 'lin_reg_1' ,'bal', verbose=verbose)
    #output2 = __model_tuning__(X_train, y_train, X_valid, y_valid, model, param_grid, 'lin_reg_1' ,'unba', verbose=verbose)


==LogisticRegression==


KeyboardInterrupt: 

In [9]:
# Balancer test

In [15]:
model1_grid= {
    'penalty': ['none', 'l2'],
    'class_weight': [None, 'balanced'],
    'n_jobs': [-1],
    'solver': ['newton-cholesky'],
    'max_iter': [500]
}
model1 = LogisticRegression
df, df_tfidf, tr = load_and_clean_data('data/train.csv', rename_dict={'tweet':'text', 'label':'output'})
X_train, X_valid, y_train, y_valid = train_test_split(df_tfidf, df['output'],test_size=0.3,random_state=random_state)

balancers = [TomekLinks(sampling_strategy='majority'), TomekLinks(sampling_strategy='not minority'), SMOTE(sampling_strategy='minority'), RandomUnderSampler(), CondensedNearestNeighbour()]
results = []
for b in balancers[:4]:
    X_train_balanced, y_train_balanced = balance_data(X_train, y_train, b)
    output = __model_tuning__(X_train_balanced, y_train_balanced, X_valid, y_valid, model1, model1_grid, 'lin_reg_1' ,'bal', verbose=False)

    results.append(f'Balancer {str(b)} : {max(output, key=lambda x: x["f1_score"])}')

print('========')


/Users/tomas.valenta/School/diploma_thesis/venv/lib/python3.10/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first place. Possible remedies are removing collinear features of X or increasing the penalization strengths.
The original Linear Algebra message was:
Ill-conditioned matrix (rcond=5.66564e-23): result may not be accurate.
  warnings.warn(
/Users/tomas.valenta/School/diploma_thesis/venv/lib/python3.10/site-packages/sklearn/linear_model/_glm/_newton_solver.py:491: LinAlgWarning: The inner solver of NewtonCholeskySolver stumbled upon a singular or very ill-conditioned Hessian matrix at iteration #1. It will now resort to lbfgs instead.
Further options are to use another solver or to avoid such situation in the first pl

In [16]:
for r in results:
    print(r)

Balancer TomekLinks(sampling_strategy='majority') : {'f1_score': 0.6240276577355228, 'confusion_matrix': array([[8793,  116],
       [ 319,  361]])}
Balancer TomekLinks(sampling_strategy='not minority') : {'f1_score': 0.6240276577355228, 'confusion_matrix': array([[8793,  116],
       [ 319,  361]])}
Balancer SMOTE(sampling_strategy='minority') : {'f1_score': 0.49530400395452306, 'confusion_matrix': array([[8067,  842],
       [ 179,  501]])}
Balancer RandomUnderSampler() : {'f1_score': 0.3555144793592113, 'confusion_matrix': array([[6920, 1989],
       [ 103,  577]])}
